In [ ]:
import pandas as pd
import lightgbm as lgb
import shap
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM


In [ ]:
import xarray as xr
import pandas as pd

ds = xr.open_dataset("df_final.nc")
df = ds.to_dataframe().reset_index()
df.head()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


train_df = df[df["timestamp"] < "2023-01-01"]  # 2013-2022  tarining
test_df = df[df["timestamp"] >= "2023-01-01"]  # 2023  test


target_column = "PCA_AQI" 
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
if target_column in numerical_features:
    numerical_features.remove(target_column)

excluded_features = ["carbon_monoxide", "PM2.5_ugm3", "ozone", "PM10_ugm3", 
                     "nitrogen_dioxide", "sulfur_dioxide"]


# Ensure numerical_features is a list of column names
numerical_features = [col for col in numerical_features if col not in excluded_features]

train_df[numerical_features] = train_df[numerical_features].fillna(train_df[numerical_features].mean())
test_df[numerical_features] = test_df[numerical_features].fillna(test_df[numerical_features].mean())

X_train = train_df[numerical_features]
y_train = train_df[target_column]
X_test = test_df[numerical_features]
y_test = test_df[target_column]

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("Train DataFrame shape:", train_df.shape)
print("Test DataFrame shape:", test_df.shape)


### Feature Selection

In [ ]:
import lightgbm as lgb
import pandas as pd
import shap
import matplotlib.pyplot as plt

lgb_train = lgb.Dataset(X_train, y_train)

params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.05, 
    "num_leaves": 31,
    "max_depth": -1,
    "feature_fraction": 0.8,  
    "bagging_fraction": 0.8,  
    "bagging_freq": 5,  
    "verbose": -1,
    "n_jobs": -1,  
}


model = lgb.train(params, lgb_train, num_boost_round=100)



In [ ]:
# Feature Importance 
import numpy as np

feature_importance = pd.DataFrame({
    'Feature': X_train.columns, 
    'Importance': model.feature_importance(importance_type='gain')  # 'gain' 
})

selected_features_df = feature_importance.nlargest(30, "Importance")
TOP_FEATURES = selected_features_df["Feature"].tolist()
TOP_FEATURES

In [ ]:
# SHAP 
explainer = shap.Explainer(model, X_train[TOP_FEATURES])
shap_values = explainer(X_test[TOP_FEATURES])


plt.figure(figsize=(10, 6))
plt.barh(feature_importance["Feature"][:30], feature_importance["Importance"][:30])
plt.xlabel("Feature Importance (LightGBM)")
plt.title("Top 30 Feature Importance")
plt.gca().invert_yaxis()
plt.show()

shap.summary_plot(shap_values, X_test[TOP_FEATURES])


selected_features_df = feature_importance.head(30)
selected_features_df

### Mixed-Effect Model

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaled_features = numerical_features + ["PCA_AQI"]  # PCA_AQI de scale edilmeli
df[scaled_features] = scaler.fit_transform(df[scaled_features])


train_df = df[df["timestamp"] < "2023-01-01"]
test_df = df[df["timestamp"] >= "2023-01-01"]



train_df['date'] = pd.to_datetime(train_df['timestamp']).dt.to_period('M')  # Aylık formatta tarih
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month

test_df['date'] = pd.to_datetime(test_df['timestamp']).dt.to_period('M')  # Aylık formatta tarih
test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month

formula = "PCA_AQI ~ temperature_2m_C_y + wind_speed_10m_ms + surface_pressure_hPa_y + total_precipitation_mm + \
total_column_ozone_y + surface_solar_radiation_downward_Wm2 + sea_salt_aerosol_1 + organic_matter_aerosol_2 + \
nitric_oxide + methane + specific_humidity_kgkg + boundary_layer_height_m + \
mean_sea_level_pressure_hPa + wind_u_component_10m_ms + wind_v_component_10m_ms + year + month + \
low_cloud_cover_percent + evaporation_mm + ethane + formaldehyde  + potential_vorticity_Km2s + relative_humidity_percent + vertical_velocity_Pas"

md = mixedlm(formula, train_df, groups=train_df["Country"])  # re_formula eklemedik, date random slope olmadı
mdf = md.fit(method='lbfgs')
print(mdf.summary())


In [ ]:
# Random effects param.
print(mdf.random_effects)


In [ ]:
# Fit Mixed Effects Model
md = sm.MixedLM.from_formula(formula, train_df, groups=train_df["Country"])
mdf = md.fit(method='lbfgs')

#  Generate Predictions on the Test Set
y_test = test_df["PCA_AQI"]  # True values
y_pred = mdf.predict(test_df)  # Model predictions

#  Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

#  Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

#  Calculate Adjusted R²
n = len(y_test)  # Number of samples
p = len(mdf.params) - 1  # Number of predictors (excluding intercept)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))


print(f" MAE: {mae:.4f}")
print(f" RMSE: {rmse:.4f}")
print(f" Adjusted R²: {adj_r2:.4f}")

print(mdf.summary())


### LSTM

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

features = ["temperature_2m_C_y", "wind_speed_10m_ms", "surface_pressure_hPa_y",
            "total_precipitation_mm", "total_column_ozone_y", "surface_solar_radiation_downward_Wm2",
            "sea_salt_aerosol_1", "organic_matter_aerosol_2", "nitric_oxide", "methane",
            "specific_humidity_kgkg", "boundary_layer_height_m", "mean_sea_level_pressure_hPa",
            "wind_u_component_10m_ms", "wind_v_component_10m_ms", "year", "month",
            "low_cloud_cover_percent", "evaporation_mm", "ethane", "formaldehyde",
            "potential_vorticity_Km2s", "relative_humidity_percent", "vertical_velocity_Pas"]

X_train = train_df[features].values
X_test = test_df[features].values
y_train = train_df["PCA_AQI"].values
y_test = test_df["PCA_AQI"].values

#  Create Sequences for LSTM (Lookback: 30 days)
lookback = 90  

def create_sequences(X, y, lookback):
    Xs, ys = [], []
    for i in range(lookback, len(X)):  # Start from `lookback` to align properly
        Xs.append(X[i - lookback : i])  # Take `lookback` previous time steps
        ys.append(y[i])  # Target is at time `i`
    return np.array(Xs), np.array(ys)

#  Create Properly Aligned Sequences
X_train_seq, y_train_seq = create_sequences(X_train, y_train, lookback)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, lookback)


In [ ]:

model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, kernel_regularizer=l2(0.01), input_shape=(lookback, len(features))),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(64, activation='relu', return_sequences=False, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(1)  # Output layer for PCA_AQI prediction
])

#  Compile Model with Learning Rate Scheduler
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='mse')

#  Learning Rate Scheduler (Reduces LR if no improvement)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=1)


history = model.fit(
    X_train_seq, y_train_seq, 
    epochs=200, batch_size=32, 
    validation_data=(X_test_seq, y_test_seq),
    callbacks=[lr_scheduler]
)


plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.legend()
plt.title("LSTM Training Loss")
plt.show()


### H-MED

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.formula.api import mixedlm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features = [
    "temperature_2m_C_y", "wind_speed_10m_ms", "surface_pressure_hPa_y",
    "total_precipitation_mm", "total_column_ozone_y", "surface_solar_radiation_downward_Wm2",
    "sea_salt_aerosol_1", "organic_matter_aerosol_2", "nitric_oxide", "methane",
    "specific_humidity_kgkg", "boundary_layer_height_m", "mean_sea_level_pressure_hPa",
    "wind_u_component_10m_ms", "wind_v_component_10m_ms", "low_cloud_cover_percent",
    "evaporation_mm", "ethane", "formaldehyde", "potential_vorticity_Km2s",
    "relative_humidity_percent", "vertical_velocity_Pas", "lat", "lon"
]
target = "PCA_AQI"

#  3. Train-Test Split
train_df = df[df["timestamp"] < "2023-01-01"].copy()
test_df = df[df["timestamp"] >= "2023-01-01"].copy()


scaler = StandardScaler()
train_df[features + [target]] = scaler.fit_transform(train_df[features + [target]])
test_df[features + [target]] = scaler.transform(test_df[features + [target]])


for df_ in [train_df, test_df]:
    df_["year"] = pd.to_datetime(df_["timestamp"]).dt.year
    df_["month"] = pd.to_datetime(df_["timestamp"]).dt.month

features += ["year", "month"]

#  Mixed-Effect Model: Random Intercept 
formula = "PCA_AQI ~ " + " + ".join(features)
md = mixedlm(formula, train_df, groups=train_df["Country"])
mdf = md.fit(method='lbfgs')

random_intercepts = {k: float(v.values[0]) if isinstance(v, pd.Series) else float(v) for k, v in mdf.random_effects.items()}
train_df["random_intercept"] = train_df["Country"].map(random_intercepts).astype(float)
test_df["random_intercept"] = test_df["Country"].map(random_intercepts).astype(float)

# Gaussian Process Regression (GPR) 
gpr_sample_size = 5000  
sampled_data = train_df.sample(n=gpr_sample_size, random_state=42)

kernel = C(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=kernel, random_state=42)
gpr.fit(sampled_data[["lat", "lon"]], sampled_data[target])

train_df["gpr_prediction"] = gpr.predict(train_df[["lat", "lon"]])
test_df["gpr_prediction"] = gpr.predict(test_df[["lat", "lon"]])
features += ["gpr_prediction"]


def to_tensor(df):
    X = torch.tensor(df[features].values, dtype=torch.float32).to(device)
    lat_lon = torch.tensor(df[["lat", "lon"]].values, dtype=torch.float32).to(device)
    year_month = torch.tensor(df[["year", "month"]].values, dtype=torch.float32).to(device)
    intercept = torch.tensor(df["random_intercept"].values, dtype=torch.float32).unsqueeze(1).to(device)
    return torch.cat((X, intercept), dim=1), lat_lon, year_month

X_train, lat_lon_train, year_month_train = to_tensor(train_df)
X_test, lat_lon_test, year_month_test = to_tensor(test_df)
y_train = torch.tensor(train_df[target].values, dtype=torch.float32).unsqueeze(1).to(device)
y_test = torch.tensor(test_df[target].values, dtype=torch.float32).unsqueeze(1).to(device)

input_dim = X_train.shape[1]
latent_dim = 16 
embed_dim = 4  

#  LSTM Diffusion 
class SpatioTemporalLSTMDiffusion(nn.Module):
    def __init__(self, input_dim, latent_dim, embed_dim):
        super(SpatioTemporalLSTMDiffusion, self).__init__()

        self.spatial_embedding = nn.Linear(2, embed_dim)
        self.temporal_embedding = nn.Linear(2, embed_dim)

        self.encoder = nn.Sequential(
            nn.Linear(input_dim + embed_dim * 2, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim)
        )

        self.lstm = nn.LSTM(latent_dim, latent_dim, num_layers=1, batch_first=True)  # 🔥 Katman sayısı azaltıldı

        self.noise_scale = nn.Parameter(torch.tensor(0.1))

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x, lat_lon, year_month, t):
        spatial_embed = self.spatial_embedding(lat_lon)
        temporal_embed = self.temporal_embedding(year_month)

        x = torch.cat([x, spatial_embed, temporal_embed], dim=1)
        z = self.encoder(x)
        
        z, _ = self.lstm(z.unsqueeze(1))  
        z = z.squeeze(1)  

        noise = torch.randn_like(z) * self.noise_scale * t
        z_noisy = z + noise  

        out = self.decoder(z_noisy)
        return out


model = SpatioTemporalLSTMDiffusion(input_dim, latent_dim, embed_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

batch_size = 512 
epochs = 5

for epoch in range(epochs):
    model.train()
    for i in range(0, X_train.shape[0], batch_size):
        optimizer.zero_grad()
        x_batch = X_train[i:i+batch_size]
        lat_lon_batch = lat_lon_train[i:i+batch_size]
        year_month_batch = year_month_train[i:i+batch_size]
        time_step = torch.rand(x_batch.shape[0], 1).to(device)
        
        y_pred = model(x_batch, lat_lon_batch, year_month_batch, time_step)
        loss = loss_fn(y_pred, y_train[i:i+batch_size])
        loss.backward()
        optimizer.step()


In [ ]:
model.eval()
with torch.no_grad():
    time_step = torch.rand(X_test.shape[0], 1).to(device)  
    y_pred_test = model(X_test, lat_lon_test, year_month_test, time_step)


y_test_np = y_test.cpu().numpy().flatten()
y_pred_np = y_pred_test.cpu().numpy().flatten()

#  Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test_np, y_pred_np)

#  Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test_np, y_pred_np))

#  Adjusted R² 
def adjusted_r2(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

n = X_test.shape[0]  
p = X_test.shape[1]  
adj_r2 = adjusted_r2(y_test_np, y_pred_np, n, p)


print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test Adjusted R²: {adj_r2:.4f}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from statsmodels.formula.api import mixedlm

# Step 1: Mixed-Effect Model - Random Intercept Calculation
formula = f"{target} ~ " + " + ".join(features)
md = mixedlm(formula, train_df, groups=train_df["Country"])
mdf = md.fit()

# Extract Random Intercepts
random_intercepts = {k: float(v.values[0]) if isinstance(v, pd.Series) else float(v) for k, v in mdf.random_effects.items()}
train_df["random_intercept"] = train_df["Country"].map(random_intercepts).astype(float)
test_df["random_intercept"] = test_df["Country"].map(random_intercepts).astype(float)

# Convert Data to Tensors
def to_tensor(df):
    X = torch.tensor(df[features].values, dtype=torch.float32).to(device)
    random_intercept = torch.tensor(df["random_intercept"].values, dtype=torch.float32).unsqueeze(1).to(device)
    return torch.cat((X, random_intercept), dim=1)

X_train = to_tensor(train_df)
X_test = to_tensor(test_df)
y_train = torch.tensor(train_df[target].values, dtype=torch.float32).unsqueeze(1).to(device)
y_test = torch.tensor(test_df[target].values, dtype=torch.float32).unsqueeze(1).to(device)

# Model Parameters
input_dim = X_train.shape[1]
latent_dim = 32  
hidden_dim = 64  
embed_dim = 8  
batch_size = 512  
learning_rate = 0.001  

#  Step 2: Hypernetwork
class HyperNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(HyperNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

#  Step 3: LSTM Forecaster
class LSTMForecaster(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMForecaster, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, weights):
        x, _ = self.lstm(x)  
        x = self.fc(x[:, -1, :])  
        return x

#  Step 4: Full HITS Model
class HITSModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(HITSModel, self).__init__()
        self.hypernetwork = HyperNetwork(input_dim, hidden_dim, latent_dim)
        self.forecaster = LSTMForecaster(latent_dim, hidden_dim, 1)

    def forward(self, x):
        weights = self.hypernetwork(x)  
        x = x.unsqueeze(1).repeat(1, 10, 1)  # Expand sequence dimension for LSTM
        output = self.forecaster(x, weights)  
        return output



### HITS

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from statsmodels.formula.api import mixedlm



def to_tensor(df):
    X = torch.tensor(df[features].values, dtype=torch.float32).to(device)
    random_intercept = torch.tensor(df["random_intercept"].values, dtype=torch.float32).unsqueeze(1).to(device)
    return torch.cat((X, random_intercept), dim=1)

X_train = to_tensor(train_df)
X_test = to_tensor(test_df)
y_train = torch.tensor(train_df[target].values, dtype=torch.float32).unsqueeze(1).to(device)
y_test = torch.tensor(test_df[target].values, dtype=torch.float32).unsqueeze(1).to(device)

#  Model Parameters
input_dim = X_train.shape[1]  # Should match Hypernetwork output
latent_dim = input_dim  # Ensure LSTM input matches Hypernetwork output
hidden_dim = 64  
embed_dim = 8  
sequence_length = 10  # Define sequence length for LSTM
batch_size = 512  
learning_rate = 0.001  

#  Hypernetwork
class HyperNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(HyperNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)  # Output must match input_dim of LSTM

#  LSTM Forecaster
class LSTMForecaster(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMForecaster, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)  # LSTM expects (batch_size, seq_len, input_dim)
        x = self.fc(x[:, -1, :])  # Use only last output step
        return x

#  Full HITS Model
class HITSModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(HITSModel, self).__init__()
        self.hypernetwork = HyperNetwork(input_dim, hidden_dim, latent_dim)
        self.forecaster = LSTMForecaster(latent_dim, hidden_dim, 1)

    def forward(self, x):
        weights = self.hypernetwork(x)  
        
        # ✅ Reshape to match LSTM input (batch_size, seq_length, input_dim)
        x = x.unsqueeze(1).repeat(1, sequence_length, 1)  
        
        output = self.forecaster(x)  
        return output

#  Initialize Model
model = HITSModel(input_dim, hidden_dim, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

#  Training Loop (With Mini-Batches)
epochs = 500
for epoch in range(epochs):
    model.train()
    for i in range(0, X_train.shape[0], batch_size):
        x_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

#  Model Evaluation
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test)

#  Performance Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_test_np = y_test.cpu().numpy().flatten()
y_pred_np = y_pred_test.cpu().numpy().flatten()

mae = mean_absolute_error(y_test_np, y_pred_np)
rmse = np.sqrt(mean_squared_error(y_test_np, y_pred_np))
r2 = r2_score(y_test_np, y_pred_np)

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")
